In [67]:
from pymongo import MongoClient
from pathlib import Path
import os
from pprint import pprint


In [68]:
client = MongoClient('mongodb://localhost:27017')
testDb = client.testDb

In [83]:
patients = testDb.patients
sessions = testDb.sessions
reports = testDb.reports

In [70]:
def fileName(filePath):
    return (os.path.splitext(os.path.split(filePath)[1]))[0]

def fileExt(filePath):
    return (os.path.splitext(os.path.split(filePath)[1]))[1]

In [77]:
p1 = os.listdir("v1.2.0/eval")
print(p1)
print(os.path.isdir(p1[0]))
print(os.path.isfile(p1[1]))
pathString = ("/"+str(p1[0]))
print(pathString)
patientDict = {}
fileNames = []
maxFileSize = 0 
maxFileName = ""
for root,dirs,files in os.walk("v1.2.0"):
    #print ("root: ",root)
    #print("\tdirs: ",dirs)
    #print("\t\t",str(files))
        
    if len(files)>0:
        for file in files:
            fileNames.append(str(root)+"/"+str(file))
            if os.path.getsize((str(root)+"/"+file)) > maxFileSize:
                maxFileSize = os.path.getsize((str(root)+"/"+file))
                maxFileName = (str(root)+"/"+file)
            else:
                continue
            
            
        #print ("\t\tFile: "+files[3])
        
print (maxFileName," : ",maxFileSize)
print (os.path.splitext(maxFileName))
print (os.path.split(maxFileName))
print (os.path.splitext(os.path.split(maxFileName)[1]))
print (fileName(maxFileName))
print (fileExt(maxFileName))

['03_tcp_ar_a', '02_tcp_le', '.DS_Store', '01_tcp_ar']
False
False
/03_tcp_ar_a
v1.2.0/eval/01_tcp_ar/00001278/s002_2003_11_17/00001278_s002_t001.edf  :  67899648
('v1.2.0/eval/01_tcp_ar/00001278/s002_2003_11_17/00001278_s002_t001', '.edf')
('v1.2.0/eval/01_tcp_ar/00001278/s002_2003_11_17', '00001278_s002_t001.edf')
('00001278_s002_t001', '.edf')
00001278_s002_t001
.edf


In [107]:
#print(fileNames[0:30])
testDb.patients.delete_many({})
testDb.sessions.delete_many({})
testDb.reports.delete_many({})

for files in fileNames[30:50]:
    print(files)
    print (files.split("/"))
    print ("File Ext: "+fileExt(files))
    print ("File Name: ",fileName(files),"\n")

patientList = []
patientSessions = []
sessionList = []
for files in fileNames:
    splitFile = files.split("/")
    #print(splitFile)
    if len(splitFile)<5:
        pass
    else:
        patientId = splitFile[3]
        sessionId = splitFile[4]
        if patientId in patientList:
            continue
        else:
            patients.insert_one({'patientId':patientId})
            patientList.append(patientId)
        if sessionId in sessionList:
            continue
        else:
            sessionList.append(sessionId)
            if patientId in patientList:
                sessions.update_one({"patientId":patientId},{'$push':{"sessionsId":sessionId}})
            else:
                sessions.insert_one({{'patientId':patientId},{'sessionId':[sessionId]}})
print (patientList)
print (len(patientList))

v1.2.0/eval/03_tcp_ar_a/00005943/s004_2009_06_29/00005943_s004_t000.tse_bi
['v1.2.0', 'eval', '03_tcp_ar_a', '00005943', 's004_2009_06_29', '00005943_s004_t000.tse_bi']
File Ext: .tse_bi
File Name:  00005943_s004_t000 

v1.2.0/eval/03_tcp_ar_a/00005943/s004_2009_06_29/00005943_s004_t000.edf
['v1.2.0', 'eval', '03_tcp_ar_a', '00005943', 's004_2009_06_29', '00005943_s004_t000.edf']
File Ext: .edf
File Name:  00005943_s004_t000 

v1.2.0/eval/03_tcp_ar_a/00005943/s004_2009_06_29/00005943_s004_t002.edf
['v1.2.0', 'eval', '03_tcp_ar_a', '00005943', 's004_2009_06_29', '00005943_s004_t002.edf']
File Ext: .edf
File Name:  00005943_s004_t002 

v1.2.0/eval/03_tcp_ar_a/00005943/s004_2009_06_29/00005943_s004_t003.edf
['v1.2.0', 'eval', '03_tcp_ar_a', '00005943', 's004_2009_06_29', '00005943_s004_t003.edf']
File Ext: .edf
File Name:  00005943_s004_t003 

v1.2.0/eval/03_tcp_ar_a/00005943/s004_2009_06_29/00005943_s004_t001.lbl_bi
['v1.2.0', 'eval', '03_tcp_ar_a', '00005943', 's004_2009_06_29', '000059

In [106]:
patientDbPrint = sessions.find({})
for patient in patientDbPrint:
    print(patient)
